In [1]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.utils.weight_optimizer as wo

In [2]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

# MLP
oof_single_17 = np.load("../artifacts/preds/l1/oof_single_17.npy")
oof_single_21 = np.load("../artifacts/preds/l1/oof_single_21.npy")

# XGB
oof_single_7 = np.load("../artifacts/preds/l1/oof_single_7.npy")

# CB
oof_single_13 = np.load("../artifacts/preds/l1/oof_single_13.npy")

# LGBM
oof_single_10 = np.load("../artifacts/preds/l1/oof_single_10.npy")

In [7]:
# Tuning
list_v1 = [
    oof_single_17,   # MLP
    oof_single_21,
    oof_single_7,   # XGB
    oof_single_13,   # CB
    oof_single_10    # LGBM
]
importlib.reload(wo)
objective = wo.create_objective(list_v1)

study = wo.run_optuna_search(
    objective,
    n_trials=500,
    direction="maximize",
    study_name="ens_v1",
    storage=url,
    sampler=optuna.samplers.TPESampler(
        n_startup_trials=100,
        seed=42)
)

[I 2025-08-05 19:43:32,337] Using an existing study with name 'ens_v1' instead of creating a new one.


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-08-05 19:43:32,655] Trial 501 finished with value: 0.9699956782288663 and parameters: {'raw_w0': 0.8642402844409877, 'raw_w1': 0.8978343356150222, 'raw_w2': 0.00032701685940223716, 'raw_w3': 0.11930111976702405, 'raw_w4': 0.018228385962963035}. Best is trial 501 with value: 0.9699956782288663.
[I 2025-08-05 19:43:32,839] Trial 502 finished with value: 0.9700004123749495 and parameters: {'raw_w0': 0.8441281735743029, 'raw_w1': 0.857713490506249, 'raw_w2': 0.07008293411330531, 'raw_w3': 0.12918776306868243, 'raw_w4': 0.07711058760006792}. Best is trial 502 with value: 0.9700004123749495.
[I 2025-08-05 19:43:33,033] Trial 503 finished with value: 0.9699799595514923 and parameters: {'raw_w0': 0.8617789064414141, 'raw_w1': 0.8543094512830754, 'raw_w2': 0.16606487197774822, 'raw_w3': 0.13465139765816106, 'raw_w4': 0.06714631905300218}. Best is trial 502 with value: 0.9700004123749495.
[I 2025-08-05 19:43:33,232] Trial 504 finished with value: 0.9699714446437309 and parameters: {'raw_

In [5]:
for oof in list_v1:
    print(oof.shape)

(750000,)
(750000,)
(750000,)
(750000, 2)
(750000,)
